In [ ]:
!pip install transformers

## ステップ1: 必要なライブラリのインストール

まず、画像処理と深度推定に必要な`transformers`ライブラリをインストールします。このライブラリにはHugging Faceの事前学習済みモデルが含まれています。

## ステップ2: Google Driveの接続とフォルダ作成

Google Driveをマウント（接続）して、画像を保存するフォルダを作成します。このフォルダには後で使用する画像をアップロードすることができます。

## ステップ3: 深度推定モデルの読み込み

ここでは、**Depth-Anything-V2-Small**という軽量な深度推定モデルを読み込みます。このモデルはHugging Faceから提供されており、CPUでも動作します。

### 使用するモデル
- **モデル名**: depth-anything/Depth-Anything-V2-Small-hf
- **特徴**: 軽量で高速、CPUでも実行可能

# 深層学習体験会: 単眼深度推定

このノートブックでは、**単眼深度推定**（Monocular Depth Estimation）を学びます。

## 単眼深度推定とは？

単眼深度推定とは、1枚の画像から各ピクセルの「奥行き」を推定する技術です。人間が2つの目で立体感を得るのとは異なり、AIは1枚の画像だけから奥行き情報を予測します。

### 応用例
- 自動運転: 障害物までの距離を推定
- ARアプリ: 3D空間の理解
- ロボティクス: 物体認識と距離計測
- 画像編集: 背景のぼかし効果

## このノートブックの流れ
1. 必要なライブラリのインストール
2. Google Driveの接続
3. 深度推定モデルの読み込み
4. サンプル画像での推論
5. 自分の画像での推論

## ステップ4: サンプル画像の読み込み

まずはインターネット上のサンプル画像（猫の画像）で深度推定を試してみます。

## ステップ5: 深度推定の実行と可視化

読み込んだ画像に対して深度推定を実行し、結果を可視化します。

### 出力される情報
- **元画像**: 入力した画像
- **深度マップ**: 色で奥行きを表現（暖色=近い、寒色=遠い）

深度マップでは、カメラに近い部分が明るく（または赤く）、遠い部分が暗く（または青く）表示されます。

---

## ステップ6: 自分の画像で試してみよう

次に、Google Driveに保存した自分の画像を使って深度推定を試します。

### 準備
1. Google Driveの`/MyDrive/images/`フォルダに画像をアップロード
2. ファイル名を`depth_estimation_1.png`にするか、下のコードのファイル名を変更してください

## ステップ7: 自分の画像で深度推定を実行

アップロードした画像に対して深度推定を実行します。結果を確認して、どの部分が近く、どの部分が遠いと判定されたか観察してみましょう。

In [ ]:
# Google Driveをマウント（接続）する
from google.colab import drive
import os

# Google Driveに接続（初回は認証が必要です）
drive.mount('/content/drive')

# 画像を保存するディレクトリのパス
images_dir = "/content/drive/MyDrive/images"

# ディレクトリが存在しない場合は作成する（exist_ok=Trueで既存の場合はエラーにしない）
os.makedirs(images_dir, exist_ok=True)

# 作成完了メッセージを表示
print("✅ フォルダ作成:", images_dir)

## (単眼)深度推定

In [ ]:
# --- モデルの読み込み ---
# 必要なライブラリをインポート
from transformers import pipeline  # Hugging Faceのパイプライン機能
from PIL import Image  # 画像処理用ライブラリ
import matplotlib.pyplot as plt  # グラフ・画像表示用
import requests  # URLから画像をダウンロードするため

# Hugging Faceの深度推定パイプラインを作成
# task: 実行するタスク（深度推定）
# model: 使用するモデル名（Depth-Anything-V2-Small）
pipe = pipeline(
    task="depth-estimation",
    model="depth-anything/Depth-Anything-V2-Small-hf"
)

In [ ]:
# --- サンプル画像の読み込み ---
# インターネット上の画像URLを指定（COCO Datasetの猫の画像）
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'

# URLから画像をダウンロードして開く
# stream=True: データを少しずつダウンロード
# .raw: 生データとして取得
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
# --- 深度推定の実行 ---
# パイプラインに画像を渡して推論を実行
result = pipe(image)

# --- 出力の中身を確認 ---
# 結果に含まれるキーとデータ型を表示
print({k: type(v) for k, v in result.items()})

# --- 深度マップ（数値データ）を取得 ---
# 推論結果から深度情報を取り出す
depth_tensor = result["predicted_depth"]
# テンソル形状を (1, H, W) から (H, W) に変換してNumPy配列化
depth_array = depth_tensor.squeeze().numpy()

# --- 結果の可視化 ---
# 図のサイズを指定（横10インチ、縦4インチ）
plt.figure(figsize=(10,4))

# 左側: 元画像を表示
plt.subplot(1,2,1)  # 1行2列の1番目
plt.title("入力画像")  # タイトル
plt.imshow(image)  # 画像を表示
plt.axis("off")  # 軸を非表示

# 右側: 深度マップを表示
plt.subplot(1,2,2)  # 1行2列の2番目
plt.title("推定された深度")  # タイトル
plt.imshow(depth_array, cmap="plasma")  # プラズマカラーマップで表示（暖色=近い、寒色=遠い）
plt.colorbar(label="深度（相対値）")  # カラーバーを追加
plt.axis("off")  # 軸を非表示

# グラフを表示
plt.show()

In [ ]:
# --- Google Drive内の画像を開く ---
# 自分がアップロードした画像のパスを指定
image_path = "/content/drive/MyDrive/images/depth_estimation_1.png"

# 画像を開いてRGB形式に変換（カラー画像として読み込む）
image = Image.open(image_path).convert("RGB")

In [ ]:
# --- 推論実行 ---
# 自分の画像に対して深度推定を実行
result = pipe(image)

# --- 出力確認 ---
# 結果に含まれるキーとデータ型を表示
print({k: type(v) for k, v in result.items()})

# --- 深度マップ（数値データ）を取得して可視化 ---
# 推論結果から深度情報を取り出す
depth_tensor = result["predicted_depth"]
# テンソル形状を (1,H,W) から (H,W) に変換してNumPy配列化
depth_array = depth_tensor.squeeze().numpy()

# --- 結果の可視化 ---
# 図のサイズを指定
plt.figure(figsize=(10,4))

# 左側: 元画像
plt.subplot(1,2,1)
plt.title("入力画像")
plt.imshow(image)
plt.axis("off")

# 右側: 深度マップ
plt.subplot(1,2,2)
plt.title("推定された深度")
plt.imshow(depth_array, cmap="plasma")  # プラズマカラーマップで可視化
plt.colorbar(label="深度（相対値）")
plt.axis("off")

# グラフを表示
plt.show()